# main.ipynb
This notebook contains some template code to help you with loading/preprocessing the data.

We start with some imports and constants.
The training data is found in the `data` subfolder.
There is also a tokenizer I've trained for you which you can use for the project.

In [ ]:
import datasets
from functools import partial
from transformers.models.deberta_v2.tokenization_deberta_v2 import DebertaV2Tokenizer as Tokenizer

TRAIN_PATH = './data/train.txt'
DEV_PATH = './data/dev.txt'
SPM_PATH = './data/tokenizer.model'
DEVICE = 'cpu'

Here are we load the dataset and tokenizer:

In [ ]:
dataset = datasets.load_dataset('text', data_files={'train': TRAIN_PATH, 'validation': DEV_PATH}) # loads the dataset
tokenizer = Tokenizer.from_pretrained(SPM_PATH) # loads the tokenizer

Next we need to tokenize the data. I've split this into 2 functions, 1 for an encoder model, 1 for a decoder model. 
These functions are written to take a dictionary of lists, and return the same. This is specifically to allow us to use the `map()` function in the `datasets` library, which we show below later. 

### Tokenization (Encoder)
For encoder tokenization, we will be doing MLM, and masking is applied randomly and dynamically during training. Right now, the function below is a preprocessing step, and thus only applied once. So we skip masking for now, and just say the input and labels are the same. 

The argument `add_special_tokens` is set to `True` (it is also true by default, but I put it here for clarity), which means that special tokens that go at the start and end of the text will be added. [CLS] and [SEP] are special tokens marking the beginning and end of the text. So, e.g. "The sky is blue." will become "[CLS] The sky is blue. [SEP]"

### Tokenization (Decoder)
For decoder tokenization, we will be doing CLM, so predicting the next token. Therefore we need to offset the labels from the inputs so that the 1st token is used to predict the 2nd, etc. See the paragraph above for an explanation of [CLS] and [SEP]. 

In [ ]:
def tokenize_encoder(examples, tokenizer):
    batch = {
        "input_ids": [],
        "labels": [],
    }

    for example in examples["text"]:
        tokens = tokenizer.encode(example, add_special_tokens=True) # will add [CLS] and [SEP]
        batch["input_ids"].append(tokens)
        batch["labels"].append(tokens)

    return batch

def tokenize_decoder(examples, tokenizer):
    batch = {
        "input_ids": [],
        "labels": [],
    }

    for example in examples["text"]:
        tokens = tokenizer.encode(example, add_special_tokens=True) # will add [CLS] and [SEP]
        batch["input_ids"].append(tokens[:-1])
        batch["labels"].append(tokens[1:])

    return batch

Now we can apply the tokenization. I show it for `tokenize_encoder` but it is the same for `tokenize_decoder`. First the function needs to only take 1 parameter, which is fine because our tokenizer is constant, so we can just apply `functools.partial`. Next, we apply `map()`, which allows fast parallel preprocessing of the dataset. 

In [ ]:
tokenize_fn = partial(tokenize_encoder, tokenizer=tokenizer) # need to make the function unary for map()
dataset = dataset.map(tokenize_fn, batched=True, num_proc=4, remove_columns=['text']) # map works with functions that return a dictionary